In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from collections import OrderedDict
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import  matplotlib.pyplot as plt

In [45]:
class Net(nn.Module):
    def __init__(self, X_train, hiddenlayer, node, Ouputnode):
        super().__init__() #nn.Module의 __init__속성 및 메소드를 불러옴
        self.X_train = X_train
        self.hiddenlayer = hiddenlayer
        self.node = node
        self.Ouputnode = Ouputnode
        ordered_dict = OrderedDict()
        ordered_dict['Linear1'] = nn.Linear(X_train.size(1) , node)
        ordered_dict['relu1'] = nn.ReLU()
        for i in range(2 , hiddenlayer+2 ):
            ordered_dict['Linear{}'.format(i)] = nn.Linear(node, node)
            ordered_dict['relu{}'.format(i)] = nn.ReLU()
        ordered_dict['Linear{}'.format(hiddenlayer+2)] = nn.Linear(node, Ouputnode)
        self.layer1  = nn.Sequential(ordered_dict)
        
        
    def forward (self, X_train):
        pred_x = self.layer1(X_train)
        
        return pred_x
    
            

In [47]:
def CNet(X_train, y_train, X_test, hiddenlayer, node, Ouputnode, epochs):
    #신경망 생성하기
    model = Net(X_train, hiddenlayer, node, Ouputnode)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    losses = []
    for epoch in range(epochs):
        optimizer.zero_grad()# optimizer의 매개변수를 0으로 만듬
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        losses.append(loss)
        loss.backward()
    
        optimizer.step()
    y_test_pred = model(X_test)  
    return y_test_pred
    

In [48]:
def RNet(X_train, y_train, X_test, hiddenlayer, node, Ouputnode, epochs):
    #신경망 생성하기
    model = Net(X_train, hiddenlayer, node, Ouputnode)
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    losses = []
    for epoch in range(epochs):
        optimizer.zero_grad() # optimizer의 매개변수를 0으로 만듬
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        losses.append(loss)
        loss.backward()
    
        optimizer.step()
        
    y_test_pred = model(X_test)  
    return y_test_pred
    
    

In [2]:
iris = load_iris()
X = iris.data
y = iris.target
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=33)

#토치 텐서로 전환
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)


In [35]:
y_train

tensor([1, 1, 2, 2, 2, 2, 2, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 2, 0, 0, 1, 2, 0, 1,
        2, 2, 1, 1, 0, 0, 2, 0, 0, 2, 1, 1, 2, 2, 2, 2, 0, 0, 1, 1, 0, 1, 2, 1,
        2, 0, 2, 0, 1, 0, 2, 1, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 1, 0, 1, 0,
        1, 1, 1, 1, 1, 0, 1, 0, 1, 2, 0, 0, 0, 0, 2, 2, 0, 1, 1, 2, 1, 0, 0, 1,
        1, 1, 0, 1, 1, 0, 2, 2, 2, 1, 2, 0, 1, 0, 0, 0, 2, 1, 2, 1, 2, 1, 2, 0])

In [3]:
print(X_train[0])

tensor([5.6000, 2.7000, 4.2000, 1.3000])


In [37]:
hiddenlayer = 3
node = 5
Ouputnode = 3
epochs = 200 